In [1]:
%matplotlib inline
# PartialState is missing in the newest transformers
#!pip install transformers==4.28.0
#!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import torch
import pandas as pd 
from typing import Any, TypeVar
from typing import List, Dict, Union
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments 
from transformers import Trainer, AutoModelForCausalLM, IntervalStrategy

In [3]:
torch.manual_seed(2137)

In [4]:
# Assign values to few params 
MODEL_NAME: str = 'EleutherAI/gpt-neo-125M'
BOS_TOKEN: str = '<|startoftext|>'
EOS_TOKEN: str = '<|endoftext|>'
PAD_TOKEN: str = '<|pad|>'

# Load tokenizer 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, bos_token = BOS_TOKEN, 
                                          eos_token = EOS_TOKEN, pad_token = PAD_TOKEN)

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


AttributeError: 'FloatProgress' object has no attribute 'style'

In [5]:
model =  AutoModelForCausalLM.from_pretrained(MODEL_NAME).cuda()
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [6]:
class PatientDiagnozeDataset(Dataset):
    
    def __init__(self, txt_list, tokenizer, max_length):
        
        self.input_ids: List = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer(BOS_TOKEN + txt + EOS_TOKEN, truncation=True, 
                                      max_length = max_length, padding = "max_length")
            self.input_ids.append(torch.tensor(encodings_dict["input_ids"]))
            self.attn_masks.append(torch.tensor(encodings_dict["attention_mask"]))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [7]:
# Load the data
#DATA_PATH = 'archive/diagnose_en_dataset.feather'
DATA_PATH = 'diagnose_en_dataset.feather'
data = pd.read_feather(DATA_PATH)
data = data['Patient'].values

In [8]:
SEQ_LEN = 1024
SAMPLE_SIZE =  int(data.shape[0] * 0.01)
_data = [el[:SEQ_LEN]  for el in data[:SAMPLE_SIZE]]


dataset = PatientDiagnozeDataset(txt_list = _data, tokenizer = tokenizer, max_length = 1024)

# Spit the data
TRAIN_SIZE = int(len(dataset) * 0.8)
train_dataset, val_dataset = random_split(dataset, [TRAIN_SIZE, len(dataset) - TRAIN_SIZE])

In [9]:
# Create output paths
os.makedirs('./results', exist_ok = True)
OUTPUT_DIR: str = './results'

In [10]:
training_args = TrainingArguments(output_dir = OUTPUT_DIR, num_train_epochs = 2, logging_steps = 5000, 
                                  save_strategy="epoch",
                                  per_device_train_batch_size=2, per_device_eval_batch_size=2, 
                                  warmup_steps=50, weight_decay=0.01, logging_dir='./logs', 
                                  evaluation_strategy="epoch",
                                 load_best_model_at_end=True)

In [11]:
_trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])})
_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.030075
2,No log,1.247455


TrainOutput(global_step=2060, training_loss=1.5178003403746967, metrics={'train_runtime': 1485.984, 'train_samples_per_second': 2.771, 'train_steps_per_second': 1.386, 'total_flos': 2151301653725184.0, 'train_loss': 1.5178003403746967, 'epoch': 2.0})

In [13]:
generated = tokenizer(BOS_TOKEN, return_tensors="pt").input_ids.cuda()

In [18]:
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                bos_token='<|startoftext|>',
                                eos_token='<|endoftext|>', pad_token='<|pad|>',
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1231 in generate        │
│                                                                                                  │
│   1228 │   │   generation_config = copy.deepcopy(generation_config)                              │
│   1229 │   │   model_kwargs = generation_config.update(**kwargs)  # All unused kwargs must be m  │
│   1230 │   │   generation_config.validate()                                                      │
│ ❱ 1231 │   │   self._validate_model_kwargs(model_kwargs.copy())                                  │
│   1232 │   │                                                                                     │
│   1233 │   │   # 2. Set generation parameters if not already defined                             │
│   1234 │   │   logits_processor = logits_processor if logits_processor is not None else LogitsP  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1109 in                 │
│ _validate_model_kwargs                                                                           │
│                                                                                                  │
│   1106 │   │   │   │   unused_model_args.append(key)                                             │
│   1107 │   │                                                                                     │
│   1108 │   │   if unused_model_args:                                                             │
│ ❱ 1109 │   │   │   raise ValueError(                                                             │
│   1110 │   │   │   │   f"The following `model_kwargs` are not used by the model: {unused_model_  │
│   1111 │   │   │   │   " generate arguments will also show up in this list)"                     │
│   1112 │   │   │   )                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: The following `model_kwargs` are not used by the model: ['bos_token', 'eos_token', 'pad_token'] (note: 
typos in the generate arguments will also show up in this list)